# Training an MT5 model for English paraphrasing

Main configuration

In [1]:
initial_finetuning = True  # this is true only at the beginning of fine-tuning. Set to False if you want to continue training from some checkpoint saved on google drive.
hf_checkpoint = 'google/mt5-small'
drive_checkpoint = ''  # e.g. '/content/drive/MyDrive/models/old-checkpoint-234/'

## Environment Setup

We need a GPU, so we check the availability:

In [2]:
!nvidia-smi

Mon May 22 21:18:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

And we install all neede libraries.

In [3]:
!pip install datasets==2.11.0 transformers==4.28.0 nltk==3.8.1 parascore==1.0.5 sentencepiece==0.1.98

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

We store checkpoints on Google Drive. After we have mounted our Google Drive, the root folder of our Drive is at `/content/drive/MyDrive/`.

In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## Data Download and Preparation

In [5]:
from datasets import load_dataset

We use our own created dataset for german-german paraphrases.

In [6]:
raw_dataset = load_dataset('yawnick/para_crawl_enen')
raw_dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/yawnick___csv/yawnick--para_crawl_enen-1825b56ac7ffdb28/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Original', 'Paraphrase'],
        num_rows: 55544
    })
    test: Dataset({
        features: ['Original', 'Paraphrase'],
        num_rows: 11532
    })
    validation: Dataset({
        features: ['Original', 'Paraphrase'],
        num_rows: 9803
    })
})

Let's store the splits separately and look at one example.

In [7]:
raw_dataset_train = raw_dataset['train']
raw_dataset_val = raw_dataset['validation']
raw_dataset_test = raw_dataset['test']
raw_dataset_train[4]

{'Original': 'Sailing is also called, to go exploring.',
 'Paraphrase': 'Sailing also means going on a voyage of discovery.'}

Now, let's prepare the data for training.

In [8]:
from transformers import T5Tokenizer, AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(hf_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Let's see how the tokenizer works:

In [10]:
s1 = raw_dataset_train[4]['Original']
s2 = raw_dataset_train[4]['Paraphrase']
print(s1)
print(s2)
inputs = tokenizer(s1, text_target=s2)
print([tokenizer.decode(id) for id in inputs['input_ids']])
inputs

Sailing is also called, to go exploring.
Sailing also means going on a voyage of discovery.
['Sail', 'ing', 'is', 'also', '', 'called', ',', 'to', 'go', 'explor', 'ing', '.', '</s>']


{'input_ids': [77703, 347, 339, 1607, 259, 13075, 261, 288, 1002, 74063, 347, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [77703, 347, 1607, 259, 12909, 259, 5846, 351, 259, 262, 35112, 304, 259, 125555, 260, 1]}

Now we create a preprocess function that turns a dataset item into a form that the model can use for training.

In [11]:
max_length = 128

# the prefix has to (dynamically) be adjusted depending on the language or when training multilingually (I think).
prefix = 'paraphrase: '

def preprocess_function(examples):
    inputs = [prefix+s1 for s1 in examples['Original']]
    targets = examples['Paraphrase']
    # most likely there will be nothing to truncate, but we still add it
    model_inputs = tokenizer(inputs, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

Now we apply the preprocessing function to the datasets.

In [12]:
tokenized_ds_train = raw_dataset_train.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_dataset_train.column_names
)
tokenized_ds_val = raw_dataset_val.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_dataset_val.column_names
)
tokenized_ds_test = raw_dataset_test.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_dataset_test.column_names
)

Map:   0%|          | 0/55544 [00:00<?, ? examples/s]

Map:   0%|          | 0/9803 [00:00<?, ? examples/s]

Map:   0%|          | 0/11532 [00:00<?, ? examples/s]

Now the data is ready.

## Model and Training Preparation

Next, the model and a Datacollator.

In [13]:
from transformers import MT5ForConditionalGeneration

Either load the pretrained model from huggingface at the beginning of fine-tuning for the first epoch, or load the model from a previous fine-tune checkooint from google drive.

In [14]:
if initial_finetuning:
  model = MT5ForConditionalGeneration.from_pretrained(hf_checkpoint)
else:
  model = MT5ForConditionalGeneration.from_pretrained(drive_checkpoint)

Next, we instantiate a DataCollator.

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Here, I'll skip the example usage of the datacollator, check it out [here](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt#data-collation).

Now, let's continue with metrics. We will use Parascore.

In [16]:
from parascore import ParaScorer

scorer = ParaScorer(lang='en', model_type='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Let's quickly go over how Parascore is used:

In [17]:
cands = ["A young person is skating.", "I like sports.", "He catches the ball.", "That's very interesting!"]
sources = ["There's a child on a skateboard.", "I like to relax.", "good morning, everyone!", "I find this interesting."]
score = scorer.free_score(cands, sources)
float(score[-1].mean())

0.5691054463386536

Now, here's the `compute_metrics` function (mostly copied from [here](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt#metrics)):

In [18]:
import numpy as np

In [19]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    print(decoded_preds[:5])
    print(decoded_labels[:5])
    
    parascore = scorer.free_score(decoded_preds, decoded_labels)
    return {'parascore': float(parascore[-1].mean())}
    

In [20]:
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

In [21]:


args = Seq2SeqTrainingArguments(
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy='steps',
    logging_steps=500,
    output_dir='/content/drive/MyDrive/models/mono-enen',  # this is where the checkpoint will be saved
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
)

In [22]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Parascore
1,1.936500,1.436945,0.853097
2,1.663700,1.341544,0.855161
3,1.572900,1.301196,0.855467
4,1.530800,1.278281,0.855545
5,1.491500,1.268282,0.855569


['Understand the different level of care you can expect from each facility.', 'A job opportunity is an opportunity to tell people what you can offer as well as what', 'Each visitor will be asked about the purpose of stay, accommodation and eventual', 'Create beautiful landscapes, terrain features, gardens and decks for your perfect outdoor', 'Securely paid hoy service for a low price!']
['Understand the other level of care you expect from each facility.', 'A job offer is an opportunity to tell people what you can offer as well as what you need.', 'Each visitor will be provided with the purpose of the stay, accommodation and possible employment.', 'Create beautiful landscapes, terrain shapes, gardens and decks for your perfect outdoor living space!', 'Safe hoy service paid at a low price!']
['Understand the different level of care you can expect from each facility.', 'A job opportunity is an opportunity to tell people what you can offer as well as what', 'Each visitor will be asked abou

TrainOutput(global_step=17360, training_loss=2.033456297843687, metrics={'train_runtime': 7839.7454, 'train_samples_per_second': 35.425, 'train_steps_per_second': 2.214, 'total_flos': 1.16031228063744e+16, 'train_loss': 2.033456297843687, 'epoch': 5.0})